# Highlights of Compiling, linking, and the C++ ecosystem

  * So far we did everything within the interactive environment.
  * To contribute to larger projects and to build optimized applications, we need to use the compiler directly
  

# C++ Compilers

Widely used compilers include:
  * GCC is the current standard in HEP
  * LLVM/Clang
  * Intel/ICC
  * Microsoft/MSVC
  
We will use GCC today. The syntax is essentially the same (some compiler options might be different)

Best practice note: Using 2 will help find bugs and practices that do not conform to the C++ standard. Each compiler is different 


# What do compilers do?

Inputs can be
   * source code 
   * header/interface files
   * already built libraries
   
Output can be
   * Object files
   * libraries 
   * executables
   
Typical usage is to build objects files, (optionally) combine them into libraries, and build an overall executable

<img src="./images/compile.png" width="500" />

# Essential options

Execution
  * ```-E``` stop after preprocessing 
  * ```-c``` stop after compilation 

Optimization
  * ```-g``` add debug symbols
  * ```-O0``` no optimization - Use ```-g -O0``` to debug
  * ```-O2``` and ```-O3``` to have the compiler (highly) optimize [```-O3``` is a good default]
  
File management
  * ```-I<directory>``` Directory in which to look for header/include files (use multiple ```-I``` for multiple paths)
  * ```-L<directory>``` Directory in which to look for (shared) libraries (use multiple ```-L``` for multiple paths)
  * ```-l<lib name>``` Library to link against (but drop the leading ```lib``` and trailing ```.so``` or ```.a```
  * ```-o<output name>``` Output file
  

# More essential options

Compiler configuration
  * ```-std=c++14``` to set the C++ standard to use (11,14,17,20....)
  * ```-fPIC``` needed when compiling for shared libraries
  
Warnings
  * ```-Wall``` get all warnings (but only a message in stdout..)
  * ```-Werror``` Any warning stops compilation
  * You also have fine-grained control over different types of warnings to enable as errors
  
Best practice: Use ```-Wall``` - warnings are more often than not a sign of a code bug

# Lets try it

  * First lets look at the environment

In [24]:
%%sh
cd /notebook/code
ls *.cpp
echo "\nsome include files too"
ls includes

hello.cpp
main.cpp

some include files too
Untitled.ipynb
hello.hpp


In [25]:
%%sh
which g++

/opt/conda/envs/2023-05-01-hsf-india-tutorial/bin/g++


In [26]:
%%sh
g++ -c hello.cpp

hello.cpp:1:10: fatal error: hello.hpp: No such file or directory
    1 | #include "hello.hpp"
      |          ^~~~~~~~~~~
compilation terminated.


CalledProcessError: Command 'b'g++ -c hello.cpp\n'' returned non-zero exit status 1.

Whoops, we need to tell the compiler where to find the include file that our code uses

In [28]:
%%sh
g++ -c hello.cpp -I./includes

And we should add recommended options

In [30]:
%%sh
g++ -c hello.cpp -I./includes -Wall -std=c++17

hello.cpp: In function 'void checkIt()':
hello.cpp:13:7: warning: unused variable 'i' [-Wunused-variable]
   13 |   int i;
      |       ^


Whoops, we found a bug. lets fix it and try again

In [33]:
%%sh
g++ -c hello.cpp -I./includes -Wall -std=c++17
ls -l hello.o

-rw-r--r-- 1 root root 15080 Apr 29 04:34 hello.o


Great. Now it works. We can also optimize our program

In [52]:
%%sh
g++ -c hello.cpp -I./includes -Wall  -std=c++17 -O3
ls -l hello.o

-rw-r--r-- 1 root root 4432 Apr 29 04:42 hello.o


Great - this is a simple program, so we can build an executable just with this

In [53]:
%%sh
g++ main.cpp hello.o -I./includes -std=c++17
ls -l a.out

-rwxr-xr-x 1 root root 16896 Apr 29 04:42 a.out


Whoops - lets give our program a better name

In [54]:
%%sh
g++ main.cpp hello.o -I./includes -o hello_world
ls -l hello_world

-rwxr-xr-x 1 root root 16896 Apr 29 04:42 hello_world


And run it

In [55]:
%%sh
./hello_world

Hello, world 0
Hello, world 1
Hello, world 2
Bonjour
Ciao
Guten Tag
Hello


We can also make a static library and link against it

In [56]:
%%sh
rm hello_world
ar rcs libhello.a hello.o
ls -l libhello.a
rm hello.o
g++ main.cpp -I./includes -lhello -L. -o hello_world
ls -l hello_world

-rw-r--r-- 1 root root 4632 Apr 29 04:42 libhello.a
-rwxr-xr-x 1 root root 16896 Apr 29 04:42 hello_world


Now lets try the same, but with a shared library

In [60]:
%%sh
rm libhello.a hello.o hello_world

In [66]:
%%sh
g++ -c hello.cpp -I./includes  -Wall -std=c++17 -O3 -fPIC 
ls -l hello.o

-rw-r--r-- 1 root root 4432 Apr 29 04:46 hello.o


Next build the shared library itself

In [71]:
%%sh
g++ hello.o -shared -o libhello.so
ls -l libhello.so

-rwxr-xr-x 1 root root 16192 Apr 29 04:47 libhello.so


And the executable

In [74]:
%%sh
g++ main.cpp -I./includes -lhello -L. -o hello_world
ls -l hello_world

-rwxr-xr-x 1 root root 16040 Apr 29 04:48 hello_world


And run it

In [76]:
%%sh
./hello_world

./hello_world: error while loading shared libraries: libhello.so: cannot open shared object file: No such file or directory


CalledProcessError: Command 'b'./hello_world\n'' returned non-zero exit status 127.

whoops - that did not work because with shared libraries need to be available/findable at run time

In [78]:
%%sh
export LD_LIBRARY_PATH=$PWD
./hello_world

Hello, world 0
Hello, world 1
Hello, world 2
Bonjour
Ciao
Guten Tag
Hello


Now it works.
  * We adjusted LD_LIBRARY_PATH at run time to find the library we just built. 

Shared libraries are the recommended approach for complex programs. This differs from static linking:
  * All ```.so``` must be available at run time.
  * No need to relink the program when a single translation unit is changed. Just recompile it and remake the corresponding shared library.
  * Large flexibly enhancement, but normally at a small computational penelty

# How to generalize this for your program?

  * For each source translation unit (```cpp``` file)
```g++ -fPIC <file> -I./includes -O3``` <br>
```g++ <list of .o files> -shared -o <output library name>``` <br>
```g++ <file with main> -l<library name> .... options```

# Automating your compile and link steps

  * Makefiles (simple but relatively hardcoded)
  * automake (relatively obsolete now)
  * cmake -- Flexible and powerful
  
**Recommendation: Use cmake**

Learning cmake is highly useful when managing libraries and programs (I need to learn it too - so I won't try to teach it) 
  * Here is a course on it ADD LINK